In [18]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.stats import zscore
import glob


In [19]:
#reading rheast and extracting districts
tamsat= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
preharvest = pd.read_csv(r'data/preharvest/Zambia_District _Maize_Forecasting_2011_2022.xlsx - 2022.csv', header=1)
postharvest=pd.read_csv('data/postharvest/postharvest_cleaned.csv')


In [20]:
# remove spaces
preharvest.columns=preharvest.columns.str.lower()
preharvest.columns=preharvest.columns.str.strip()
preharvest.columns=preharvest.columns.str.replace(' ', '', regex=True)#remove space in between

# remove all white spaces, turn to lower case and
preharvest.district=preharvest.district.str.lower()
preharvest.district = preharvest.district.str.strip()
preharvest.district=preharvest.district.str.replace('-', '', regex=True)#remove hyphen
# preharvest.district=preharvest.district.str.replace(' ', '', regex=True)#removes spaces anywhere

# remove spaces in other columns as well
preharvest.planted_ha = preharvest.planted_ha.str.strip()
preharvest.planted_ha=preharvest.planted_ha.str.replace(',', '', regex=True)#removes spaces anywhere

preharvest.harvested_ha = preharvest.harvested_ha.str.strip()
preharvest.harvested_ha=preharvest.harvested_ha.str.replace(',', '', regex=True)#removes spaces anywhere

preharvest.production_mt = preharvest.production_mt.str.strip()
preharvest.production_mt=preharvest.production_mt.str.replace(',', '', regex=True)#removes spaces anywhere

In [21]:
# drop province
preharvest=preharvest.drop('province', axis=1)

In [22]:
# clean preharvest survey data
print(len(preharvest))
#drop duplicate inputs
preharvest = preharvest.drop_duplicates()
print(len(preharvest))
preharvest

996
996


,district,planted_ha,harvested_ha,production_mt,yield(mt/ha),year
0,chibombo,69254,52256,148888,2.15,2011
1,kabwe,5526,5084,25637,4.64,2011
2,kapirimposhi,48651,37962,103335,2.12,2011
3,mkushi,34536,28538,130000,3.76,2011
4,mumbwa,35944,29544,95568,2.66,2011
...,...,...,...,...,...,...
991,mongu,12433,9711,11025,0.89,2022
992,senanga,19309,10295,11132,0.58,2022
993,sesheke,17756,10282,18585,1.05,2022
994,shangombo,20788,15981,24396,1.17,2022


In [23]:
# convert appropriate columns to numeric
preharvest.planted_ha=pd.to_numeric(preharvest.planted_ha)
preharvest.harvested_ha=pd.to_numeric(preharvest.harvested_ha)
preharvest.production_mt=pd.to_numeric(preharvest.production_mt)
preharvest['yield(mt/ha)']=pd.to_numeric(preharvest['yield(mt/ha)'])

In [25]:

#calculate yield
preharvest = preharvest.groupby(['district','year'],as_index=False).sum()
preharvest['yield'] = preharvest['production_mt'] / preharvest['planted_ha']
preharvest['yield_h'] = preharvest['production_mt'] / preharvest['harvested_ha']
preharvest



,district,year,planted_ha,harvested_ha,production_mt,yield(mt/ha),yield,yield_h
0,chadiza,2011,21879,21293,42951,1.96,1.963115,2.017142
1,chadiza,2012,23894,21453,51340,2.15,2.148657,2.393138
2,chadiza,2013,24901,23476,50022,2.01,2.008835,2.130772
3,chadiza,2014,25395,22957,61135,2.41,2.407364,2.663022
4,chadiza,2015,25379,21825,51345,2.02,2.023129,2.352577
...,...,...,...,...,...,...,...,...
893,zambezi,2018,4682,3941,7595,1.62,1.622170,1.927176
894,zambezi,2019,5885,3990,6973,1.18,1.184877,1.747619
895,zambezi,2020,7193,6031,12153,1.69,1.689559,2.015089
896,zambezi,2021,6350,5533,18935,2.98,2.981890,3.422194


In [26]:
# #calculate z scores of yield and flag any anomalies
df1 = []
# # preharvest = list(preharvest.reset_index())
for d in list(preharvest.district.unique()):
    df = preharvest.loc[preharvest['district']==d].copy()
    df['zyield'] = zscore(df['yield'])
    df1.append(df)
    df2=pd.concat(df1)


In [27]:
# add a flag to the dataframe

df2['flag'] = 0
df2['flag'].loc[(((df2['zyield']>1.96).copy())|((df2['zyield']<-1.96).copy()))]= 1
df2['flag'].loc[(((df2['zyield']>2.58).copy())|((df2['zyield']<-2.58).copy()))] = 2
preharvest_district=df2

C:\Users\kasit\AppData\Local\Temp\ipykernel_13372\701027530.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['flag'].loc[(((df2['zyield']>1.96).copy())|((df2['zyield']<-1.96).copy()))]= 1
C:\Users\kasit\AppData\Local\Temp\ipykernel_13372\701027530.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['flag'].loc[(((df2['zyield']>2.58).copy())|((df2['zyield']<-2.58).copy()))] = 2


In [28]:
# weedout outliers if any

In [29]:
# # save distric level data to csv
preharvest_district.to_csv('data/preharvest/zambia_preharvest_district_data_clean.csv', index=False)

In [30]:
# read out shapefile and rename columns appropriately
# read data
zambia_district_shp=gpd.read_file('data/shp/zmb_district_cleaned.shp')

# rename preharvest column to same name as shapefile
preharvest_district.rename(columns={'district':'district_b'}, inplace=True)

# merge
preharvest_shp=pd.merge(zambia_district_shp,preharvest_district, on='district_b')

In [31]:
# aggregate to province level
preharvest_province=preharvest_shp.groupby(['province_b','year'], 
                                           as_index=False).agg({'production_mt':'sum','planted_ha': 'sum', 'harvested_ha': 'sum'})

In [32]:
# calculate province level yield 
preharvest_province['yield']=preharvest_province['production_mt']/preharvest_province['planted_ha']
preharvest_province['yield_h']=preharvest_province['production_mt']/preharvest_province['harvested_ha']

preharvest_province

,province_b,year,production_mt,planted_ha,harvested_ha,yield,yield_h
0,central,2011,558493,211185,169991,2.644568,3.285427
1,central,2012,494215,184049,156385,2.685236,3.160246
2,central,2013,478734,217001,159372,2.206137,3.003878
3,central,2014,723762,256406,217439,2.822719,3.328575
4,central,2015,484723,273008,164809,1.775490,2.941120
...,...,...,...,...,...,...,...
115,western,2018,75110,93447,68387,0.803771,1.098308
116,western,2019,22758,132120,22961,0.172252,0.991159
117,western,2020,95478,110144,76460,0.866847,1.248731
118,western,2021,96235,110896,83640,0.867795,1.150586


In [33]:
# check if file exists and if not save

# Give the filename you wish to save the file to
filename = 'data/preharvest/zambia_preharvest_province_data_clean.csv'

# Use this function to search for any files which match your filename
files_present = glob.glob(filename)


# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    preharvest_province.to_csv('data/preharvest/zambia_preharvest_province_data_clean.csv',index=False)
else:
    print('WARNING: This file already exists!') 

In [34]:
# overwrite csv
preharvest_province.to_csv('data/preharvest/zambia_preharvest_province_data_clean.csv', index=False)